In [660]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os
import pickle
import yaml
from tqdm import tqdm
from src.rpg_trajectory_evaluation.align_trajectory import *

In [661]:

def getSyncedMap(timestamps_first,timestamps_second,threshold=0.05,add_nans=False): 
    '''
    This function gets two list of time stamps and returns a list of synchronizing maps
    [ ...[first_index,corresponding_synchronizing_second_index]...]. if there are no indices
    in the second timestamp lists that is close enough to the indices in the first list (dt<threshold),
    nan will be used to indicate the situation.
    '''
    map_list=[]
    for i in range(len(timestamps_first)):
        corresponding_second_index=np.argmin(np.abs(timestamps_second -timestamps_first[i]))
        min_dt=np.min(np.abs(timestamps_second -timestamps_first[i]))
        if min_dt<threshold:
            map_list.append((i,corresponding_second_index))
        elif add_nans:
             map_list.append((i,np.nan))

    return np.array(map_list).astype(np.long)

def downSample(pi, pj, n=100):
    idx = np.array([i*n for i in range(min(pi.shape[0], pj.shape[0])//n)])
    return pi[idx, ...], pj[idx,...]

def syncPaths(stamp1, stamp2, pos1, pos2):
    maps = get_synced_samples(stamp1, stamp2, threshold=0.05, add_nans=False)
    P1 = pos1[maps[:,0],...]
    P2 = pos2[maps[:,1],...]
    return stamp2[maps[:,1]], P1, P2, maps

def umeyamaMatch(points1, points2):
    A, B = downSample(points1, points2, n=2)
    s, R, t = align_umeyama(A,B)
#     s=0.49863936219059213
    ext_params = {'S':s, 'R':R, 'T':t}
    aligned_gt =( s * R @ points2.T + t.reshape(3,1)).T
    return aligned_gt, ext_params

    

## Extract The Data From the CSV File

In [662]:
# data_root = 'expriments/mars_small_z_pos_large_rot_cov_simulated_IMU_stamp_smaller_cal_cov_TC21_2/'
data_root = 'expriments/final/old_cov_and_fixed_mars_prop_issue/'

cable_len_df = pd.read_csv(os.path.join(data_root,'cable_len.csv'))
gt_pose_df = pd.read_csv(os.path.join(data_root,'gt_pose.csv'))
mars_center_pose_df = pd.read_csv(os.path.join(data_root,'mars_center_pose.csv'))
mars_imu_pose_df = pd.read_csv(os.path.join(data_root,'mars_imu_pose.csv'))

raw_imu_df = pd.read_csv(os.path.join(data_root,'raw_imu.csv'))
solver_pose_df = pd.read_csv(os.path.join(data_root,'sover_pose.csv'))
spc_pose_df = pd.read_csv(os.path.join(data_root,'spc_pose.csv'))

#Extracting Time Stamps
cable_len_stamp = cable_len_df.loc[:,'stamp'].to_numpy()
gt_pose_stamp = gt_pose_df.loc[:,'stamp'].to_numpy()
mars_center_pose_stamp = mars_center_pose_df.loc[:,'stamp'].to_numpy()
mars_imu_pose_stamp = mars_imu_pose_df.loc[:,'stamp'].to_numpy()
raw_imu_stamp = raw_imu_df.loc[:,'stamp'].to_numpy()
solver_pose_stamp = solver_pose_df.loc[:,'stamp'].to_numpy()
spc_pose_stamp = spc_pose_df.loc[:,'stamp'].to_numpy()

In [663]:
cal_state_df = pd.read_csv(os.path.join(data_root,'mars_calstate_pose.csv'))

#Extracting Time Stamps
cal_state_stamp = cal_state_df.loc[:,'stamp'].to_numpy()
cal_state_pos   = cal_state_df.loc[:,'px':'pz'].to_numpy()
cal_state_rot   = cal_state_df.loc[:,'qw':'qz'].to_numpy()

In [658]:
import pyquaternion
d = 4/180*np.pi
q_original = pyquaternion.Quaternion(0.3827,0,0,-0.9239)
ax = q_original.axis
angle = q_original.angle
q_perturbed = pyquaternion.Quaternion(axis = ax, angle=angle-d)
q_perturbed

Quaternion(0.41470004345974654, 0.0, 0.0, -0.9099581715411342)

In [664]:
%matplotlib
fig, ax = plt.subplots(2,1,sharex=True)

ax[0].plot(cal_state_stamp,cal_state_pos)

ax[0].plot(cal_state_stamp,cal_state_pos)
ax[0].legend(['x', 'y','z'])
ax[0].grid(True)
ax[0].set_title(f'cal state pos')


ax[1].plot(cal_state_stamp,cal_state_rot)
ax[1].legend(['w', 'x', 'y','z'])
ax[1].grid(True)
ax[1].set_title(f'cal state rot')


plt.tight_layout()
# plt.savefig('/home/rouholla/large_var.pdf')

Using matplotlib backend: TkAgg


In [665]:
cal_state_pos[-1]

array([-0.1610039 ,  0.03469276,  0.07875326])

In [584]:
cal_state_pos[-1]

array([-0.13227496,  0.04949875,  0.0794818 ])

In [666]:
cal_state_df = pd.read_csv(os.path.join(data_root,'mars_bias_state.csv'))

#Extracting Time Stamps
bias_stamp = cal_state_df.loc[:,'stamp'].to_numpy()
bias_accel   = cal_state_df.loc[:,'bax':'baz'].to_numpy()
bias_gyro   = cal_state_df.loc[:,'bgx':'bgz'].to_numpy()

In [667]:
fig, ax = plt.subplots(2,1,sharex=True)

ax[0].plot(bias_stamp,bias_accel)
ax[0].legend(['x', 'y','z'])
ax[0].grid(True)
ax[0].set_title(f'accel_bias')


ax[1].plot(bias_stamp,bias_gyro)
ax[1].legend(['x', 'y','z'])
ax[1].grid(True)
ax[1].set_title(f'gyro bias')


plt.tight_layout()
# plt.savefig('/home/rouholla/large_var.pdf')

## SPC Pose Vs GT Pose

In [668]:
spc_pos = spc_pose_df.loc[:,'px':'pz'].to_numpy()
gt_pos = gt_pose_df.loc[:,'px':'pz'].to_numpy()

In [669]:
map = getSyncedMap(spc_pose_stamp, gt_pose_stamp, threshold=0.05)
map

array([[    0,     0],
       [    1,     2],
       [    2,     3],
       ...,
       [32403, 45399],
       [32404, 45400],
       [32405, 45402]])

In [670]:
spc_pos_sync = spc_pos[map[:,0],...]
gt_pos_sync = gt_pos[map[:,1],...]

In [671]:
spc_pos_sync_aligned, _ = umeyamaMatch(gt_pos_sync, spc_pos_sync)

In [672]:
%matplotlib
idx = [i*10 for i in range(spc_pos_sync_aligned.shape[0]//10)]
ax = plt.axes(projection = '3d')
ax.plot(gt_pos_sync[:,0], gt_pos_sync[:,1], gt_pos_sync[:,2],'k')
ax.plot(spc_pos_sync_aligned[idx,0], spc_pos_sync_aligned[idx,1], spc_pos_sync_aligned[idx,2],'*')

Using matplotlib backend: TkAgg


## Solver Vs GT Pose

In [673]:
solver_pose = solver_pose_df.loc[:,'px':'pz'].to_numpy()
gt_pos = gt_pose_df.loc[:,'px':'pz'].to_numpy()

In [674]:
map = getSyncedMap(gt_pose_stamp, solver_pose_stamp, threshold=0.05)
map

array([[    0,     0],
       [    1,     0],
       [    2,     0],
       ...,
       [45400,  2495],
       [45401,  2495],
       [45402,  2495]])

In [675]:
solver_pose_stamp_sync = solver_pose_stamp[map[:,1],...]

In [676]:
solver_pose_sync = solver_pose[map[:,1],...]
gt_pos_sync = gt_pos[map[:,0],...]

In [677]:
vars = []
for i in range(-200,50):
    if i == 0:
        P0 = gt_pos_sync
        P1 = solver_pose_sync
    elif i>0:
        P0 = gt_pos_sync[i:,...]
        P1 = solver_pose_sync[:-i,:]   
    else:
        P0 = gt_pos_sync[:i:,...]
        P1 = solver_pose_sync[-i:,:]   

    s1, R1, t1 = align_umeyama(P0,P1)
    P1_aligned =( s1 * R1 @ P1.T + t1.reshape(3,1)).T

    var = np.linalg.norm(np.var(P0-P1_aligned, axis=0))
    vars.append([i, var])
vars = np.array(vars)
plt.plot(vars[:,0], vars[:,1])
min_var = min(vars[:,1])
min_offset = int(vars[np.argmin(vars[:,1]),0])
plt.plot(min_offset, min_var,'r*')
print(min_offset)


6


In [678]:
if min_offset == 0:
    P0 = gt_pos_sync
    P1 = solver_pose_sync
    
elif min_offset>0:
    P0 = gt_pos_sync[min_offset:,...]
    P1 = solver_pose_sync[:-min_offset,:]
else:
    P0 = gt_pos_sync[:min_offset:,...]
    P1 = solver_pose_sync[-min_offset:,:] 

s1, R1, t1 = align_umeyama(P0[15000:,...],P1[15000:,...])
solver_pose_sync_aligned = ( s1 * R1 @ P1.T + t1.reshape(3,1)).T
err = np.std(P0[1000:]-solver_pose_sync_aligned[1000:], axis = 0)
np.linalg.norm(err)

0.03593280960687225

In [679]:
solver_pose_sync_aligned, _ = umeyamaMatch(gt_pos_sync, solver_pose_sync)

In [598]:
%matplotlib
plt.figure()
ax = plt.axes(projection = '3d')
ax.plot(gt_pos_sync[:,0], gt_pos_sync[:,1], gt_pos_sync[:,2],'k')
ax.plot(solver_pose_sync_aligned[:,0], solver_pose_sync_aligned[:,1], solver_pose_sync_aligned[:,2],'*')

Using matplotlib backend: TkAgg


In [644]:
plt.plot(solver_pose_sync_aligned-gt_pos_sync)
err = np.std(solver_pose_sync_aligned-gt_pos_sync, axis = 0)
np.linalg.norm(err)

ValueError: operands could not be broadcast together with shapes (49876,3) (51898,3) 

In [600]:
np.savetxt('SOLVER_VS_GT_GT_POSES_ALIGNED.csv', gt_pos_sync, delimiter=',')
np.savetxt('SOLVER_VS_GT_SOLVER_POSES_ALIGNED.csv', solver_pose_sync_aligned, delimiter=',')

## MARS_Center Vs GT Pose (And Correlation Based Time Alignment)

In [680]:
import pyquaternion
mars_center_pos = mars_center_pose_df.loc[:,'px':'pz'].to_numpy()
mars_center_qs = mars_center_pose_df.loc[:,'qw':'qz'].to_numpy()
mars_center_rots = np.vstack([pyquaternion.Quaternion(mars_center_qs[i,:]).rotation_matrix.reshape(1,3,3) for i in range(mars_center_qs.shape[0])])

gt_pos = gt_pose_df.loc[:,'px':'pz'].to_numpy()

In [681]:
map = getSyncedMap(gt_pose_stamp, mars_center_pose_stamp, threshold=0.1)
map

array([[     0,      0],
       [     1,      0],
       [     2,      0],
       ...,
       [ 45400, 106226],
       [ 45401, 106226],
       [ 45402, 106226]])

In [682]:
mars_center_pos_sync = mars_center_pos[map[:,1],...]
mars_center_stamp_sync = mars_center_pose_stamp[map[:,1],...]
gt_pos_sync = gt_pos[map[:,0],...]
gt_stamp_sync = gt_pose_stamp[map[:,0],...]

In [683]:
# plt.plot(mars_center_pos_sync[:,0])
# plt.plot(gt_pos_sync[:,0])
plt.plot(mars_center_pos[:,0])


In [684]:
%matplotlib
plt.plot(mars_center_pos_sync-gt_pos_sync)
np.std(mars_center_pos_sync-gt_pos_sync, axis = 0)

Using matplotlib backend: TkAgg


array([0.02843226, 0.02668514, 0.02216386])

In [685]:
mars_center_pos_sync_aligned, p = umeyamaMatch(gt_pos_sync, mars_center_pos_sync)
p

{'S': 0.9999973345178987,
 'R': array([[ 0.9999636 , -0.00601884,  0.00604709],
        [ 0.00600659,  0.99997988,  0.00204102],
        [-0.00605925, -0.00200463,  0.99997963]]),
 'T': array([-0.00905461, -0.02910296, -0.00251227])}

In [631]:
%matplotlib
plt.figure
idx = [i*20 for i in range(mars_center_pos_sync.shape[0]//20)]
ax = plt.axes(projection = '3d')
ax.plot(gt_pos_sync[:,0], gt_pos_sync[:,1], gt_pos_sync[:,2],'k')
ax.plot(mars_center_pos_sync_aligned[idx,0], mars_center_pos_sync_aligned[idx,1], mars_center_pos_sync_aligned[idx,2],'*')


Using matplotlib backend: TkAgg


In [686]:
vars = []
for i in range(-200,50):
    if i == 0:
        P0 = gt_pos_sync
        P1 = mars_center_pos_sync
    elif i>0:
        P0 = gt_pos_sync[i:,...]
        P1 = mars_center_pos_sync[:-i,:]   
    else:
        P0 = gt_pos_sync[:i:,...]
        P1 = mars_center_pos_sync[-i:,:]   

    s1, R1, t1 = align_umeyama(P0,P1)
    P1_aligned =( s1 * R1 @ P1.T + t1.reshape(3,1)).T

    var = np.linalg.norm(np.var(P0-P1_aligned, axis=0))
    vars.append([i, var])
vars = np.array(vars)
plt.plot(vars[:,0], vars[:,1])
min_var = min(vars[:,1])
min_offset = int(vars[np.argmin(vars[:,1]),0])
plt.plot(min_offset, min_var,'r*')
print(min_offset)


7


In [687]:
if min_offset == 0:
    P0 = gt_pos_sync
    gt_stamp_sync_aligned = gt_stamp_sync[min_offset:,...]
    P1 = mars_center_pos_sync
    mars_center_pos_sync_aligned_stamp = mars_center_stamp_sync[:,...]
    
elif min_offset>0:
    P0 = gt_pos_sync[min_offset:,...]
    gt_stamp_sync_aligned = gt_stamp_sync[min_offset:,...]
    P1 = mars_center_pos_sync[:-min_offset,:]
    mars_center_pos_sync_aligned_stamp = mars_center_stamp_sync[:-min_offset]
else:
    P0 = gt_pos_sync[:min_offset:,...]
    gt_stamp_sync_aligned = gt_stamp_sync[:min_offset:,...]

    P1 = mars_center_pos_sync[-min_offset:,:] 
    mars_center_pos_sync_aligned_stamp = mars_center_stamp_sync[-min_offset:] 
s1, R1, t1 = align_umeyama(P0[15000:,...],P1[15000:,...])
mars_center_pos_sync_aligned = ( s1 * R1 @ P1.T + t1.reshape(3,1)).T
err = np.std(P0[6500:50800]-mars_center_pos_sync_aligned[6500:50800], axis = 0)
np.linalg.norm(err)

0.0392535558927485

In [688]:
plt.plot(mars_center_pos_sync_aligned)

In [614]:
with open('alignment_params.pckl','wb') as f:
    pickle.dump({'R':R1, 'S':s1, 'T':t1, 'time_sift':7},f)

In [615]:
np.savetxt('MARS_VS_GT_GT_POSES_ALIGNED.csv', P0, delimiter=',')
np.savetxt('MARS_VS_GT_MARS_POSES_ALIGNED.csv', mars_center_pos_sync_aligned, delimiter=',')


In [689]:
idx = {'X':0, 'Y':1, 'Z':2}
fig, ax = plt.subplots(3,1,sharex=True)

for i,name in enumerate(['X','Y','Z']):
    ax[i].plot(gt_stamp_sync_aligned,P0[:,idx[name]])
    ax[i].plot(mars_center_pos_sync_aligned_stamp,mars_center_pos_sync_aligned[:,idx[name]])
#     ax[i].plot(mars_center_pos_sync_aligned_stamp,P1[:,idx[name]])

    ax[i].plot(solver_pose_stamp_sync, solver_pose_sync[:,idx[name]])
    ax[i].legend(['gt', 'mars','solver'])
    ax[i].grid(True)
    ax[i].set_title(f'{name} Axis')
plt.tight_layout()
# plt.savefig('/home/rouholla/large_var.pdf')

In [520]:
axis = 'Z'
fig, ax = plt.subplots(2,1,sharex=True)

ax[0].plot(gt_stamp_sync_aligned,P0[:,2])
ax[0].plot(mars_center_pos_sync_aligned_stamp,mars_center_pos_sync_aligned[:,2])

ax[0].plot(solver_pose_stamp_sync, solver_pose_sync[:,2])
ax[0].legend(['gt', 'mars','solver'])
ax[0].grid(True)
ax[0].set_title(f'{axis} Axis')


ax[1].plot(cal_state_stamp,cal_state_pos)
ax[1].legend(['x', 'y','z'])
ax[1].grid(True)
ax[1].set_title(f'{axis} Bias')


plt.tight_layout()
# plt.savefig('/home/rouholla/large_var.pdf')

In [260]:
axis = 'Z'
fig, ax = plt.subplots(2,1,sharex=True)

ax[0].plot(gt_stamp_sync_aligned,P0[:,2])
ax[0].plot(mars_center_pos_sync_aligned_stamp,mars_center_pos_sync_aligned[:,2])

ax[0].plot(solver_pose_stamp_sync, solver_pose_sync[:,2])
ax[0].legend(['gt', 'mars','solver'])
ax[0].grid(True)
ax[0].set_title(f'{axis} Axis')


ax[1].plot(cable_len_stamp[0:-1], cable_len_stamp[1:]-cable_len_stamp[0:-1])
ax[1].grid(True)
ax[1].set_title(f'{axis} Bias')


plt.tight_layout()
# plt.savefig('/home/rouholla/large_var.pdf')

In [259]:
axis = 'Z'
l = cable_len_df.loc[:,'l1':'l4'].to_numpy()
fig, ax = plt.subplots(2,1,sharex=True)

ax[0].plot(gt_stamp_sync_aligned,P0[:,2])
ax[0].plot(mars_center_pos_sync_aligned_stamp,mars_center_pos_sync_aligned[:,2])

ax[0].plot(solver_pose_stamp_sync, solver_pose_sync[:,2])
ax[0].legend(['gt', 'mars','solver'])
ax[0].grid(True)
ax[0].set_title(f'{axis} Axis')


ax[1].plot(spc_pose_stamp[0:-1], (spc_pose_stamp[1:]-spc_pose_stamp[0:-1])*100)
ax[1].plot(spc_pose_stamp[0:], spc_pos[:,0],'*')

ax[1].grid(True)
ax[1].set_title(f'{axis} Bias')


plt.tight_layout()
# plt.savefig('/home/rouholla/large_var.pdf')

In [241]:
plt.plot(gt_stamp_sync_aligned-gt_stamp_sync_aligned[0],P0[:,0])
plt.plot(mars_center_pos_sync_aligned_stamp-gt_stamp_sync_aligned[0],mars_center_pos_sync_aligned[:,0])
plt.grid(True)

# Gyro Based Time Alignment

## GT Stamps

In [43]:
gyr = raw_imu_df.loc[:,'gx':'gz'].to_numpy()
accel = raw_imu_df.loc[:,'ax':'az'].to_numpy()
gt_pos = gt_pose_df.loc[:,'px':'pz'].to_numpy()
gt_pose_stamp = gt_pose_df.loc[:,'stamp'].to_numpy()

### Generate IMU data Simulated Stamps

In [33]:
1/((gt_pose_stamp[-1]-gt_pose_stamp[0])/len(gt_pose_stamp))

174.91816869261706

In [44]:
1/((raw_imu_stamp[-1]-raw_imu_stamp[0])/len(raw_imu_stamp))

400.0043369635826

In [45]:
%matplotlib
pol = np.polyfit([i for i in range(raw_imu_stamp.shape[0])], raw_imu_stamp, 1)
m2, b2 = pol
simulated_imu_stamps = np.stack([m2*i + b2 for i in range(raw_imu_stamp.shape[0])])
plt.plot(simulated_imu_stamps)
plt.plot(raw_imu_stamp)

Using matplotlib backend: TkAgg


In [65]:
from liegroups import SO3
import pyquaternion
gt_qs = gt_pose_df.loc[:,'qw':'qz'].to_numpy()
gt_rots = np.vstack([pyquaternion.Quaternion(gt_qs[i,:]).rotation_matrix.reshape(1,3,3) for i in range(gt_qs.shape[0])])
deltaR = gt_rots[:-1,...].transpose(0,2,1)@gt_rots[1:,...]
omega = [SO3.from_matrix(deltaR[i,...]).log()*175 for i in range(deltaR.shape[0])]
omega = np.stack(omega)

In [66]:
from scipy import signal

def filterForceSensor(f, fs = 100, fc = 10):
    c = 2*fc/fs
    b, a = signal.butter(2, Wn = c)
    return signal.filtfilt(b, a, f, padlen=150)


filtered_omega_x = filterForceSensor(omega[:,0], fs=175, fc = 5)
filtered_omega_y = filterForceSensor(omega[:,1], fs=175, fc = 5)
filtered_omega_z = filterForceSensor(omega[:,2], fs=175, fc = 5)
filtered_omega = np.vstack([filtered_omega_x,filtered_omega_y,filtered_omega_z]).T
filtered_omega_mag = np.linalg.norm(filtered_omega, axis=-1)
gyro_mag = np.linalg.norm(gyr, axis=-1)

In [67]:
plt.plot(gt_pose_stamp[0:-1], filtered_omega_mag)
plt.plot(simulated_imu_stamps, gyro_mag)

In [71]:
map = getSyncedMap(gt_pose_stamp, simulated_imu_stamps, threshold=0.02)

In [69]:
filtered_omega_mag_sync = filtered_omega_mag[map[:-100,0],]
gyro_mag_sync = gyro_mag[map[:-100,1],]

In [72]:
plt.plot(filtered_omega_mag_sync)
plt.plot(gyro_mag_sync)

## Cable Length Stamps

In [73]:
cable_len_stamp = cable_len_df.loc[:,'stamp'].to_numpy()
plt.plot(cable_len_stamp)

In [74]:
1/((cable_len_stamp[-1]-cable_len_stamp[0])/len(cable_len_stamp))

124.84376998211964

In [80]:
# plt.plot(1/(cable_len_stamp[0:-1]-cable_len_stamp[1:]))
_ = plt.plot((cable_len_stamp[0:-1]-cable_len_stamp[1:]))

In [77]:
plt.plot(gt_stamp_sync_aligned,P0[:,idx[name]])
plt.plot(mars_center_pos_sync_aligned_stamp,mars_center_pos_sync_aligned[:,idx[name]])
# plt.plot(1/(cable_len_stamp[0:-1]-cable_len_stamp[1:]))


In [20]:
raw_imu_df_filtered_stamp = pd.concat([pd.DataFrame(data = simulated_imu_stamps, columns=['filetered_stamps']) , raw_imu_df], axis=1)
raw_imu_df.to_csv(os.path.join(data_root, 'raw_imu_plus_filtered_stamps.csv'))

In [21]:
integrated = []
R = np.eye(3)
for i in range(3000):
    integrated.append(SO3(R).log())
    R=R@SO3.exp((gyr_sync[i+53900,...]-bias_gyro[-1])/400).as_matrix()

NameError: name 'SO3' is not defined

In [316]:
plt.plot(integrated)

In [317]:
integrated2 = []
R = np.eye(3)
for i in range(3000):
    integrated2.append(SO3(gt_rots_sync[54000].T @ gt_rots_sync[54000+i]@SO3.exp(integrated[i]).as_matrix()).log())

In [318]:
# plt.plot(integrated)
plt.plot(integrated2)

In [324]:
plt.hist(gt_pose_stamp[1:]-gt_pose_stamp[0:-1])

(array([4.534e+04, 4.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00]),
 array([2.71797180e-05, 4.00391102e-02, 8.00510406e-02, 1.20062971e-01,
        1.60074902e-01, 2.00086832e-01, 2.40098763e-01, 2.80110693e-01,
        3.20122623e-01, 3.60134554e-01, 4.00146484e-01]),
 <BarContainer object of 10 artists>)

In [49]:
k=simulated_imu_stamps.shape[0]/filtered_omega_mag.shape[0]
k

2.2866173296330556

In [222]:
ts=np.arange((filtered_omega_mag.shape[0]*k))/k

In [223]:
from scipy.interpolate import interp1d
f = interp1d(np.arange(filtered_omega_mag.shape[0]),filtered_omega_mag)
g=np.vstack([f(t) for t in ts[0:-2].tolist()])

In [230]:
# plt.plot(g)
# plt.plot(gyro_mag)
cor = np.correlate(gyro_mag[:103815][12000:13500], g[:103815,0][12000:13500],mode = 'full')
plt.plot(cor)

In [232]:
s=1500-np.argmax(cor)
s

92

In [234]:
# plt.plot(g[35:][0:10000,0]-gyro_mag[:-35][0:10000])
plt.plot(g[s:][0:20000,0])
plt.plot(gyro_mag[:-s][0:20000])

# plt.plot(gyro_mag[:-35],'.')
# plt.grid(True)

In [184]:
plt.plot(gt_pose_stamp[1:0]-gt_pose_stamp[0:1])

In [337]:
plt.plot(raw_imu_stamp,'.')
plt.plot([raw_imu_stamp[0]+i/400 for i in range(raw_imu_stamp.shape[0])],'.')